# IMPORTING LIBRARIES

In [232]:
!pip install textblob


In [234]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [246]:
import re
from textblob import TextBlob
from typing import List, Optional

# Preprocess Review Text

In [249]:
def preprocess(text: str) -> str:
    text = text.lower()                                 # Convert to lowercase
    text = re.sub(r"[^\w\s]", "", text)                 # Remove punctuation
    text = re.sub(r"\s+", " ", text)                    # remove whitespace
    return text.strip()

# NLP Sentiment (Text-Based)

In [252]:
def get_sentiment_from_text(text: str) -> str:
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0.1:
        return "positive"
    elif polarity < -0.1:
        return "negative"
    else:
        return "neutral"

# Sentiment Classification (from star rating)

In [255]:
def get_sentiment_from_rating(rating: int) -> str:
    if rating >= 4:
        return "positive"
    elif rating == 3:
        return "neutral"
    else:
        return "negative"

# Topic Extraction

In [294]:
def extract_topics(text: str) -> List[str]:
    topic_keywords = {
        "wait_time": ["wait", "delay", "long queue", "waiting", "slow"],
        "staff_attitude": ["rude", "impolite", "shouted", "disrespectful", "unfriendly", "nonchalant", "care"],
        "medication": ["drug", "pill", "prescription", "medication", "dose", "tablet"],
        "cost": ["expensive", "bill", "cost", "money", "price"]
    }

    found_topics = []
    for topic, keywords in topic_keywords.items():
        for word in keywords:
            if word in text:
                found_topics.append(topic)
                break  # only add topic once
    return found_topics


# Urgency Flagging

In [261]:
def flag_urgent(text: str) -> bool:
    urgent_keywords = [
        "wrong drug", "bleeding", "dying", "emergency",
        "critical", "injury", "pain", "severe", "unconscious", "collapsed"
    ]
    return any(word in text for word in urgent_keywords)

# Full Analysis Function

In [285]:
def analyze_feedback(text: Optional[str] = None, rating: Optional[int] = None) -> dict:
    # Case 1: No input at all
    if not text and rating is None:
        return {"error": "No input text or rating provided."}

    result = {}
    clean_text = preprocess(text) if text else ""

    # Case 2: Use NLP sentiment if text is given (regardless of rating)
    if text and clean_text.strip():  # ensure text is not just spaces
        nlp_sentiment = get_sentiment_from_text(clean_text)
        result["sentiment"] = nlp_sentiment
        
    # Case 3: If no text (or just empty spaces), fallback to rating sentiment
    elif rating is not None:
        rating_sentiment = get_sentiment_from_rating(rating)
        result["sentiment"] = rating_sentiment
   # Case 4: Topics are only gotten for negative sentiments and if a topic for a rating is not gotten it is flag unidentified
    if text and clean_text.strip():
        if result["sentiment"] == "negative":
            topics = extract_topics(clean_text)
            result["topics"] = topics if topics else 'Unidentified'
        result["urgent_flag"] = flag_urgent(clean_text)

    return result

# Example

In [304]:
star_rating = 2
text = "The Doctor cared so much about me"
print(analyze_feedback(text = 'text', rating = star_rating))

{'sentiment': 'neutral', 'urgent_flag': False}
